In [1]:
%pip install pandas_profiling

Note: you may need to restart the kernel to use updated packages.


'c:\program' is not recognized as an internal or external command,
operable program or batch file.


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
from datetime import datetime

%matplotlib inline

## Question: is death rate and recovery rate stable over time? difference between countries?

In [5]:
### death rate = df_death / df_confirmed
### recovery rate = df_recovered / df_confirmed

In [8]:
df_death['Province/State'] == df_confirmed['Province/State']

0      True
1      True
2      True
3      True
4      True
      ...  
71     True
72     True
73     True
74    False
75    False
Name: Province/State, Length: 76, dtype: bool

In [12]:
df_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  53 non-null     object 
 1   Country/Region  76 non-null     object 
 2   Lat             76 non-null     float64
 3   Long            76 non-null     float64
 4   1/22/20         76 non-null     int64  
 5   1/23/20         76 non-null     int64  
 6   1/24/20         76 non-null     int64  
 7   1/25/20         76 non-null     int64  
 8   1/26/20         76 non-null     int64  
 9   1/27/20         76 non-null     int64  
 10  1/28/20         76 non-null     int64  
 11  1/29/20         76 non-null     int64  
 12  1/30/20         76 non-null     int64  
 13  1/31/20         76 non-null     int64  
 14  2/1/20          76 non-null     int64  
 15  2/2/20          76 non-null     int64  
 16  2/3/20          76 non-null     int64  
 17  2/4/20          76 non-null     int64

In [31]:
df_confirmed_grouped = df_confirmed.groupby(by = 'Country/Region').sum().nlargest(5,columns = '1/22/20')

In [32]:
df_death_grouped  = df_death.groupby(by = 'Country/Region').sum().nlargest(5,columns = '1/22/20')

In [35]:
df_confirmed_grouped.fillna(0)
df_death_grouped.fillna(0)

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Mainland China,1033.98047,3458.52786,17,18,26,42,56,82,131,133,...,1111,1116,1368,1520,1662,1765,1863,2002,2114,2236
Australia,-134.08070,587.79820,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Belgium,50.50390,4.46990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cambodia,12.56570,104.99100,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Canada,135.92080,-283.74950,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1. Read data

In [42]:
df_confirmed = pd.read_csv('novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
df_death = pd.read_csv('novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv')
df_recovered = pd.read_csv('novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv')

In [43]:
profile = ProfileReport(df_confirmed, title="Pandas Profiling Report")

In [44]:
profile.to_widgets()

C:\Users\Andre\AppData\Roaming\Python\Python38\site-packages\pandas_profiling\visualisation\plot.py:154: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)


ValueError: ('widget type not understood', 'select')

## 2. Create tidy dataframes

In [ ]:
df_confirmed['Province/State'].fillna(' ', inplace=True)
df_death['Province/State'].fillna(' ', inplace=True)
df_recovered['Province/State'].fillna(' ', inplace=True)

In [ ]:
df_confirmed.set_index(['Province/State', 'Country/Region'], inplace=True)
df_death.set_index(['Province/State', 'Country/Region'], inplace=True)
df_recovered.set_index(['Province/State', 'Country/Region'], inplace=True)

In [ ]:
# drop latitude and longitude since I'm not going to do visualization
df_confirmed.drop(labels=['Lat','Long'], inplace=True, axis=1)

df_confirmed.head()

In [ ]:
# calculate death rate and recovery rate using reported numbers

date_cols = df_confirmed.columns
df_death_rate = np.divide(df_death[date_cols], df_confirmed[date_cols]).fillna(method='ffill')
df_recovery_rate = np.divide(df_recovered[date_cols], df_confirmed[date_cols]).fillna(method='ffill')

df_remaining = df_confirmed - df_death - df_recovered

Initial inspection of raw data indicate some regions have super high death rate, I'm suspecting it's due to a small number of confirmed cases, hence high variance.

In [ ]:
high_death_rate = df_death_rate.index[df_death_rate.max(axis=1) > 0.2]
high_death_rate

In [ ]:
df_confirmed.loc[high_death_rate]

Data above confirmed the speculation that high death rate is due to small sample. We can add a filter to control for sample size.

The number will only be plotted when there is more than 10 confirmed cases.

In [ ]:
# plot death rate
fig, ax = plt.subplots(figsize=(16,6))
df_death_rate[df_confirmed > 10].T.plot(ax=ax, ylim=(0, 0.1), title='Death Rate Over Time')
df_death_rate.loc['Hubei','Mainland China'].plot(ax=ax, ylim=(0, 0.1), lw=4, color='r')
plt.xlabel('Date')
plt.ylabel('Death Rate')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  

In [ ]:
# plot recovery rate
fig, ax = plt.subplots(figsize=(16,6))
df_recovery_rate[df_confirmed > 10].T.plot(ax=ax, ylim=(0, 1), title='Recovery Rate Over Time')
df_recovery_rate.loc['Hubei','Mainland China'].plot(ax=ax, ylim=(0, 1), lw=4, color='r')
plt.xlabel('Date')
plt.ylabel('Recovery Rate')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  

From figures above we can see Hubei's death rate is relatively stable around 3%, while recovery rate first dropped in late January and slowly increased early Febuary.

## 3. Align outbreak onset
As COVID-19 is spreading to different regions at different time, it might be a better view to align different regions by days since infection. Here I define the day when a region has more than 5 confirmed cases as Day 0 of infection, and create relative `Day since Infection` to plot by.

In [ ]:
# helper functions for data preprocessing

def convert_to_ts(df):
    copy = df.copy()
    copy.index = [' '.join(idx) for idx in copy.index.values]
    ts = copy.T
    ts.index = pd.to_datetime(ts.index)
    return ts

# create new time series with aligned day 0
def get_outbreak_start(ts, min_threshold=5):
    sub_ts = ts[ts >= min_threshold]
    if len(sub_ts) < 1:
        return None, None
    else:
        day0 = sub_ts.index[0]
        sub_ts.index = (sub_ts.index - pd.to_datetime(day0)).days
        return sub_ts, day0
    
def align_day0(ts, min_threshold=5):
    sub_ts_list = []
    day0_dict = {}
    for col in ts.columns:
        sub_ts, day0 = get_outbreak_start(ts[col], min_threshold)
        sub_ts_list.append(sub_ts)
        day0_dict[col] = day0
    return pd.concat(sub_ts_list, axis=1), day0_dict

In [ ]:
ts = convert_to_ts(df_confirmed)
ts_confirmed, day0_dict = align_day0(ts, min_threshold=10)

In [ ]:
# plot aligned # of cases
fig, ax = plt.subplots(figsize=(16,6))
ts_confirmed.drop('Hubei Mainland China', axis=1).plot(ax=ax, ylim=(0, 2000), title='Confirmed case Over Time')
plt.xlabel('Days since first 10 confirmed cases')
plt.ylabel('Number of confirmed cases - Outside Hubei Province')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  
# plot Hubei on a secondary axis as it has a much bigger scale than other regions
ts_confirmed['Hubei Mainland China'].plot(ax=ax, secondary_y=True, lw=3, color='r')
ax.right_ax.set_ylabel('Number of confirmed cases - Hubei Province')

In [ ]:
ts_confirmed.max().nlargest(5)

## 4. View for death rate and recovery rate after alignment

In [ ]:
ts_d = convert_to_ts(df_death.drop(labels=['Lat','Long'],axis=1))
ts_r = convert_to_ts(df_recovered.drop(labels=['Lat','Long'],axis=1))

In [ ]:
ts_death = align_day0(ts_d, min_threshold=0)
ts_recovered = align_day0(ts_r, min_threshold=0)

In [ ]:
def align_day0_with_dict(ts, day0_dict, min_threshold=0):
    ts = ts.copy()
    sub_ts_list = []
    
    for col in ts.columns:
        try:
            ts[col].index = (ts[col].index - pd.to_datetime(day0_dict[col])).days
            sub_ts_list.append(ts[col])
        except:
            continue
    return pd.concat(sub_ts_list, axis=1)


In [ ]:
ts_death = align_day0_with_dict(ts_d, day0_dict, min_threshold=0).loc[0:]
ts_recovered = align_day0_with_dict(ts_r, day0_dict, min_threshold=0).loc[0:]

In [ ]:
ts_death_rate = np.divide(ts_death, ts_confirmed)
ts_recovered_rate = np.divide(ts_recovered, ts_confirmed)

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
ts_death_rate.plot(ax=ax, ylim=(0, 0.15), title='Death rate by infection day')
ts_death_rate['Hubei Mainland China'].plot(ax=ax, ylim=(0, 0.15), lw=4, color='r')
plt.xlabel('Days since first 10 confirmed cases')
plt.ylabel('Death Rate')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  

In [ ]:
ts_death_rate.max().nlargest(5)

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
ts_recovered_rate.plot(ax=ax, ylim=(0, 1), title='Recovery rate by infection day')
ts_recovered_rate['Hubei Mainland China'].plot(ax=ax, ylim=(0, 1), lw=4, color='r')
plt.xlabel('Days since first 10 confirmed cases')
plt.ylabel('Recovery Rate')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))  

In [ ]:
ts_recovered_rate.max().nlargest(5)

Generally, we can see a slight decreasing trend in death rate and increasing trend in recovery rate in most regions. Hubei province has slightly higher death rate and lower recovery rate compared to other regions. This is potentially due to the outbreak overwhelming the hospitals and medical facilities. For other regions, death rate stablized after roughly 10 days since first reported death.

Precautions have been taken in otehr regions in China, as well as rest of the world. With better preparation and strict policy on quarantine, rest of China is doing no worse than other regions in the world. Due to the difference in onset, we will need more data over time to conduct a more accurate comparison.